## Замечания для клевера:
Актуально:
* Можно улучшить обработку текстовых данных (можно попробовать в сыром виде запихать в кэтбуст)
* Если лемматизация будет работать долго (есть такая вероятность) - можно поменять на стеммер

Старое:
* Вариант с тем, что в одной колонке название характеристики, в другой значение будет работать плохо (вот пример), т.к. модели без разницы на порядок следования колонок
* Правильно ли я понимаю, что все колонки кроме ХК 1 и целевой имеют тип данных String?
* **Как предсказывать строки с пустыми значениями во всех колонках ХК? (может их сразу откидывать)?**
* Названия первой колонки должны быть всегда одинаковые
* !!!Важно!!! Будем заменять числовые факторы на категориальные, если в них маленькое количество уникальных значений или одно значение встречается очень часто
* Были ошибки в названиях колонок: 'ХК_ка т_01'

**Проблемы, решение которых нужно будет автоматизировать:**
* Несбалансированность классов
* Пропуски в данных
* Автоматическое кодирование текстовых столбцов

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from loguru import logger

### Работа с данными

In [3]:
df = pd.read_excel('data/paper_classificator_data.xlsx')

In [4]:
df.head(1)

,ID класса (ТАРГЕТ),Наименование терминального класса,Код родительского класса,Наименование родительского класса,Историческое наименование,ХК_Кат_01,Значение ХК_Кат_01,ХК_Кат_02,Значение ХК_Кат_02,ХК_Кат_03,Значение ХК_Кат_03,ХК_Стр_01,Значение ХК_Стр_01,ХК_Числ_01,Значение ХК_Числ_01,ХК_Числ_02,Значение ХК_Числ_02,ХК_Числ_03,Значение ХК_Числ_03
0,12326143.0,"Бумага для офисной техники листовая цветная, А4",01.15.01,БУМАГА,"Бумага д/принтера цветная IQ Color, А4, 80г/м2...",Производитель,IQ Color,Формат,A4,Цвет,розовый,NaN,NaN,Листов в пачке,100.0,"Плотность, г/м2",80.0,NaN,NaN


In [5]:
target_df = df['ID класса (ТАРГЕТ)']
factors_df = df.drop('ID класса (ТАРГЕТ)', axis = 1)

#### Работаем с типами данных столбцов

In [6]:
factors_df.columns

Index(['Наименование терминального класса', 'Код родительского класса',
       'Наименование родительского класса', 'Историческое наименование',
       'ХК_Кат_01', 'Значение ХК_Кат_01', 'ХК_Кат_02', 'Значение ХК_Кат_02',
       'ХК_Кат_03', 'Значение ХК_Кат_03', 'ХК_Стр_01', 'Значение ХК_Стр_01',
       'ХК_Числ_01', 'Значение ХК_Числ_01', 'ХК_Числ_02',
       'Значение ХК_Числ_02', 'ХК_Числ_03', 'Значение ХК_Числ_03'],
      dtype='object')

In [9]:
import re

def format_column_types(columns: list):
    '''
    Обрабатывает названия колонок из массива columns.
    Возвращает словарь с парами: название колонки - ее тип данных  
    '''
    column_types_dict = {}
    for column in columns:
        type_pattern = r'ХК_([^_]+)_.*'
        if column[0:2] == 'ХК':
            column_types_dict[column] = 'Кат'
        elif column[0:8]=='Значение':
            column_type = re.findall(type_pattern, column)[0]
            column_types_dict[column] = column_type
        else:
            column_types_dict[column] = 'Стр'
            
            
    return column_types_dict

In [10]:
def check_number_to_categorical(column: str, factor: pd.Series):
    logger.info(f'Начинаем проверку численного фактора {column}\n')
    logger.debug(f'Размер фактора:{factor.size}')
    logger.debug(f'Количество уникальных значений: {factor.drop_duplicates().size}')
    logger.debug(f'Процент заполненности фактора: {factor[factor.notnull()].size / factor.size * 100}%')
    popular_value = pd.DataFrame(factor.value_counts().sort_values(ascending=False).head(1)/factor[factor.notnull()].size*100)
    popular_value.columns = ['Частота']
    logger.debug(f'Cамое частое значение фактора: \n{popular_value}')
    
    if float(popular_value.iloc[0])>=50:
        logger.info(f'Переводим числовой фактор {column} в категориальный')
        return True


column_types_dict = format_column_types(factors_df.columns)
for column in column_types_dict.keys():
    if column_types_dict.get(column) == 'Стр':
        factors_df[column] = factors_df[column].astype(str)
    elif column_types_dict.get(column) == 'Булево':
        factors_df[column] = factors_df[column].astype(bool)
    elif column_types_dict.get(column) == 'Числ':
        if check_number_to_categorical(column, factors_df[column]):
            column_types_dict[column] = 'Кат'
            factors_df[column] = factors_df[column].astype(object)
        else:   
            factors_df[column] = factors_df[column].astype(float)
    elif column_types_dict.get(column) == 'Кат':
        logger.debug(f'Категориальный фактор: {column}')
        #todo ДОДЕЛАТЬ преобразование категориальных колонок (пока не делаем, т.к. возможно будет catboost)


2023-10-05 16:15:28.325 | DEBUG    | __main__:<module>:28 - Категориальный фактор: ХК_Кат_01
2023-10-05 16:15:28.326 | DEBUG    | __main__:<module>:28 - Категориальный фактор: Значение ХК_Кат_01
2023-10-05 16:15:28.327 | DEBUG    | __main__:<module>:28 - Категориальный фактор: ХК_Кат_02
2023-10-05 16:15:28.328 | DEBUG    | __main__:<module>:28 - Категориальный фактор: Значение ХК_Кат_02
2023-10-05 16:15:28.329 | DEBUG    | __main__:<module>:28 - Категориальный фактор: ХК_Кат_03
2023-10-05 16:15:28.330 | DEBUG    | __main__:<module>:28 - Категориальный фактор: Значение ХК_Кат_03
2023-10-05 16:15:28.331 | DEBUG    | __main__:<module>:28 - Категориальный фактор: ХК_Стр_01
2023-10-05 16:15:28.333 | DEBUG    | __main__:<module>:28 - Категориальный фактор: ХК_Числ_01
2023-10-05 16:15:28.334 | INFO     | __main__:check_number_to_categorical:2 - Начинаем проверку численного фактора Значение ХК_Числ_01

2023-10-05 16:15:28.336 | DEBUG    | __main__:check_number_to_categorical:3 - Размер фактора

In [11]:
column_types_dict

{'Наименование терминального класса': 'Стр',
 'Код родительского класса': 'Стр',
 'Наименование родительского класса': 'Стр',
 'Историческое наименование': 'Стр',
 'ХК_Кат_01': 'Кат',
 'Значение ХК_Кат_01': 'Кат',
 'ХК_Кат_02': 'Кат',
 'Значение ХК_Кат_02': 'Кат',
 'ХК_Кат_03': 'Кат',
 'Значение ХК_Кат_03': 'Кат',
 'ХК_Стр_01': 'Кат',
 'Значение ХК_Стр_01': 'Стр',
 'ХК_Числ_01': 'Кат',
 'Значение ХК_Числ_01': 'Кат',
 'ХК_Числ_02': 'Кат',
 'Значение ХК_Числ_02': 'Кат',
 'ХК_Числ_03': 'Кат',
 'Значение ХК_Числ_03': 'Числ'}

#### Кодируем строковые переменные
Возможные варианты:
* bag_of_words - пока остановимся на нем
* tf_idf 

In [31]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [56]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def text_preprocessing(text):
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "  and token.strip() not in punctuation]
    return tokens

def text_feature_preprocessing(text_feature):
    '''
    Функция преобразования текстовых факторов
    - Переводим в нижний регистр
    - Удаляем знаки препинания
    - Удаляем стоп слова
    - Проводим лемматизацию
    '''
    processed_feature = []
    text_feature = text_feature.replace(r'[^\w\s]',' ', regex=True).replace(r'\s+',' ', regex=True).str.lower()
    '''for text in text_feature.values:
        tokens = mystem.lemmatize(text)
        tokens = [token for token in tokens if token not in russian_stopwords and token != " "  and token.strip() not in punctuation]
    
        processed_feature.append(tokens)
    '''
    processed_text_feature = text_feature.apply(text_preprocessing)
    return processed_text_feature




In [59]:
#print(f"Before:\n {factors_df['Историческое наименование'][0:20]}")
processed_historical_name = text_feature_preprocessing(factors_df['Историческое наименование'][0:20])
#print(f"After:\n {processed_historical_name}")

In [60]:
vectorizer = CountVectorizer()
vectorizer.fit(processed_historical_name)
bow_model = vectorizer.transform()

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
print(vectorizer.vocabulary_)

{'бумага': 337, 'принтера': 483, 'цветная': 569, 'iq': 253, 'color': 218, 'а4': 318, '80г': 183, 'м2': 416, 'pi25': 283, 'розовый': 492, '100л': 9, 'op174': 278, 'фламинго': 551, 'ye23': 313, 'желтый': 378, 'zg34': 314, 'лимонно': 413, 'печати': 469, 'офисной': 453, 'техники': 538, 'colorcode': 219, 'intensive': 252, 'микс': 428, 'цветов': 571, 'по': 478, '20л': 96, 'or43': 279, 'оранжевый': 448, 'техникиcolorcode': 539, 'creative': 225, 'креатив': 403, '80': 182, '100': 7, 'цв': 564, 'х20': 561, 'интенсив': 385, 'a4': 200, 'снегурочка': 524, 'белизна': 329, '146': 49, 'cie': 215, '500л': 148, 'mondi': 268, 'пач': 461, '96': 198, 'сыктывкар': 535, 'data': 227, 'copy': 223, 'svetocopy': 305, 'класс': 391, 'canon': 214, 'black': 208, 'lable': 257, 'extra': 238, 'premium': 286, 'label': 256, '8169b011aa': 185, '8169b001aa': 184, 'офтех': 456, 'red': 293, 'professional': 289, '172cie': 68, 'шт': 577, 'epson': 237, 'струй': 533, 's041256': 295, '167г': 61, 'матов': 419, '50л': 149, 'hp': 24

#### Заполняем пропуски в данных
В зависимости от типа данных колонки заполняем пропуски по-разному:
*   Стр -  т.к. переводим строки в числа, то пропущенные значение пусть будут = 0
*   Числ - #todo По умолчанию = 0. Если присутствует значение, количество которого в заполненных строках >=50% => то фактор станет категориальным, а не численным. 
*   Булево - #todo будем считать, что у нас всегда такие столбцы отвечают на вопрос: "Есть что-то? - Да/Нет". Если нет ответа => Нет
*   Кат - 'Emptyclass'

In [1]:
factors_df.isna().sum()

NameError: name 'factors_df' is not defined

In [19]:
new_factors_df = factors_df.copy()
for column in new_factors_df.columns:
    if column_types_dict.get(column) == 'Кат':
        new_factors_df.loc[new_factors_df[column].isna(), column] = 'EmptyValue'
    elif column_types_dict.get(column) == 'Числ':
        new_factors_df.loc[new_factors_df[column].isna(), column] = 0
    #elif column_types_dict.get(column) == 'Стр':
        # Строковых факторов не должно остаться, т.к мы переведем их в числовые
        #raise Exception('В датасете присутствуют строковые факторы!')
    elif column_types_dict.get(column) == 'Булево':
        new_factors_df.loc[new_factors_df[column].isna(), column] = 0


In [20]:
new_factors_df.isna().sum()

Наименование терминального класса     0
Код родительского класса              0
Наименование родительского класса     0
Историческое наименование             0
ХК_Кат_01                             0
Значение ХК_Кат_01                    0
ХК_Кат_02                             0
Значение ХК_Ка т_02                  55
ХК_Кат_03                             0
Значение ХК_Кат_03                    0
ХК_Стр_01                             0
Значение ХК_Стр_01                    0
ХК_Числ_01                            0
Значение ХК_Числ_01                   0
ХК_Числ_02                            0
Значение ХК_Числ_02                   0
ХК_Числ_03                            0
Значение ХК_Числ_03                   0
dtype: int64

#### Удалим бесполезные факторы которые состоят из 1 уникального значения

In [21]:
#Делаем это после кодирования категориалььных переменных
for column in factors_df.columns:
    unique_values_count = factors_df[column].drop_duplicates().size
    if unique_values_count == 1:
        new_factors_df = factors_df.drop(column, axis = 1)


In [1]:
new_factors_df.info()

NameError: name 'new_factors_df' is not defined

In [25]:
new_factors_df['Наименование терминального класса'].value_counts()

Наименование терминального класса
nan                                                                         98
Бумага для офисной техники листовая, А4                                     56
Картон цветной, А4, 10цветов, упак                                          28
Картон цветной, А4, 10 цветов, 10 листов, упак                              16
Бумага для офисной техники листовая цветная, А4                             11
Картон цветной, А4, 8 цветов, 8 листов, упак                                 9
Картон цветной, А3, 8 цветов, 8 лист, упак                                   7
Калька для копировальных работ листовая                                      7
Картон цветной, А4, 5 цветов, 5 листов, упак                                 7
Картон гофрированный A4, 5л, упак                                            6
Фотобумага листовая                                                          6
Бумага перфорированная однослойная                                           6
Фотобумага рулонна